: 

In [2]:
pip install rasterio

     ---------------------------------------- 22.9/22.9 MB 4.7 MB/s eta 0:00:00
  Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached snuggs-1.4.7-py3-none-any.whl (5.4 kB)
  Using cached affine-2.4.0-py3-none-any.whl (15 kB)
  Using cached click-8.1.7-py3-none-any.whl (97 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: C:\Users\agast\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
pip install pil

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pil (from versions: none)
ERROR: No matching distribution found for pil

[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: C:\Users\agast\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import os
import math
from itertools import product
import rasterio as rio
from PIL import Image 
from rasterio import windows

In [5]:
def get_tiles(ds, tile_width, tile_height, overlap):
    nols, nrows = ds.meta['width'], ds.meta['height']
    xstep = tile_width - overlap
    ystep = tile_height - overlap
    for x in range(0, nols, xstep):
        if x + tile_width > nols:
            x = nols - tile_width
        for y in range(0, nrows, ystep):
            if y + tile_height > nrows:
                y = nrows - tile_height
            window = windows.Window(x, y, tile_width, tile_height)
            transform = windows.transform(window, ds.transform)
            yield window, transform

in_path = 'Raw/'
input_filename = '5m.tif'

out_path = 'Output_10m/'
output_filename = 'tile_{}-{}.tif'

tile_width = 100
tile_height = 100
overlap = 0

with rio.open(os.path.join(in_path, input_filename)) as src:
    metadata = src.meta.copy()
    print(metadata)

    for window, transform in get_tiles(src, tile_width, tile_height, overlap):
        metadata['transform'] = transform
        metadata['width'], metadata['height'] = window.width, window.height
        out_filepath = os.path.join(out_path, output_filename.format(window.col_off, window.row_off))
        
        with rio.open(out_filepath, 'w', **metadata) as dst:
            dst.write(src.read(window=window))

{'driver': 'GTiff', 'dtype': 'uint16', 'nodata': None, 'width': 17958, 'height': 16346, 'count': 3, 'crs': CRS.from_epsg(32644), 'transform': Affine(5.0, 0.0, 333658.94308370305,
       0.0, -5.0, 1664427.5)}


In [9]:
def generate_dirty(data_path):
    """
    Generate the X values by downsampling clean imagery.

    Args:
        data_path:

    """
    input_train_path = data_path 
   

    output_train_path = "5m_400px"
 

    
  
    for file in os.listdir(str(input_train_path)):
        
        if(os.path.splitext(str(input_train_path +'/'+ file))[1].lower()=='.tif' or os.path.splitext(str(input_train_path +'/'+ file))[1].lower()=='tiff'):
            img = Image.open(str(input_train_path +'/'+ file))
            # change the size according to the input
            temp = img.resize((400, 400), Image.BILINEAR) 
            temp.save(str(output_train_path +'/'+ file))
        else :
            continue

generate_dirty("Output_10m")


UnidentifiedImageError: cannot identify image file 'Output_10m/tile_0-0.tif'